<a href="https://colab.research.google.com/github/kushb2/Techno-Funda/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# ==============================================================================
# PROJECT: INSTITUTIONAL SNIPER (v4.1 - LIVE DATA EDITION)
# OBJECTIVE: Scan stocks for Institutional Footprints using REAL NSE DATA
# ==============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import time

# Try importing nsepython
try:
    from nsepython import nse_quote
    print("✅ NSE Python Library Detected.")
except ImportError:
    nse_quote = None
    print("⚠️ NSEPython not found. Please install: pip install nsepython")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
class Config:
    # --- YOUR WATCHLIST ---
    # Real stocks to test the live connection
    WATCHLIST = [

        "NETWEB.NS"
    ]

    # --- Analysis Thresholds ---
    MIN_ROE = 10.0
    MAX_DEBT = 1.5

    # --- The Sniper Scope (Institutional Footprint) ---
    VOL_LOOKBACK = 20
    Z_SCORE_LIMIT = 2.0      # Volume Anomaly Threshold
    DELIVERY_LIMIT = 40.0    # Minimum Delivery %

    SIMULATE_NSE = False     # <--- WE ARE GOING LIVE! 🔴

# ==============================================================================
# 2. DATA ENGINE
# ==============================================================================
def fetch_data(ticker):
    """Fetches Fundamentals and 6 Months of Price History via yfinance."""
    try:
        stock = yf.Ticker(ticker)
        # Fast info fetch
        info = stock.fast_info
        # Detailed info for fundamentals (can be slow, using fallback if needed)
        fund_info = stock.info

        hist = stock.history(period="6mo")
        if hist.empty: return None, None
        return fund_info, hist
    except:
        return None, None

def get_delivery(ticker):
    """
    Fetches REAL Delivery % from NSE using the 'trade_info' endpoint.
    """
    # 1. Check if we are simulating
    if Config.SIMULATE_NSE:
        sym = ticker.replace(".NS", "")
        if sym in ["NETWEB", "INFY"]: return 65.0
        if sym in ["PAYTM", "SUZLON"]: return 15.0
        return 35.0

    # 2. Real Data Fetch
    if nse_quote is None: return 0.0

    sym = ticker.replace(".NS", "")
    try:
        # We use the specific section where delivery data lives
        data = nse_quote(sym, section="trade_info")

        if data and 'securityWiseDP' in data:
            delivery_data = data['securityWiseDP']
            deliv_pct = float(delivery_data.get('deliveryToTradedQuantity', 0))
            return deliv_pct

        return 0.0 # Return 0 if data missing (don't crash)

    except Exception as e:
        # If NSE blocks us or times out, just return 0 for this stock
        print(f"   (⚠️ Data fetch failed for {sym})")
        return 0.0

# ==============================================================================
# 3. ANALYSIS LOGIC (The Brain)
# ==============================================================================
def analyze_fundamentals(info):
    """Returns a Warning string if unsafe."""
    warnings = []

    # Handle different yfinance data structures safely
    try:
        roe = info.get('returnOnEquity', 0)
        if roe is None: roe = 0
        roe = roe * 100

        debt = info.get('debtToEquity', 0)
        if debt is None: debt = 0
        debt = debt / 100

        if roe < Config.MIN_ROE: warnings.append(f"Low ROE ({roe:.1f}%)")
        if debt > Config.MAX_DEBT: warnings.append(f"High Debt ({debt:.1f})")
    except:
        return "⚠️ Data Missing"

    return ", ".join(warnings) if warnings else "✅ Safe"

def analyze_trend(hist):
    """Returns Trend Status based on 50/200 SMA."""
    try:
        period = 200 if len(hist) > 200 else 50
        sma = hist['Close'].rolling(window=period).mean().iloc[-1]
        price = hist['Close'].iloc[-1]

        if price > sma: return "✅ Uptrend"
        return "⚠️ Downtrend"
    except:
        return "Err"

def check_institutional_action(hist, ticker):
    """
    LAYER 3: Volume Z-Score + Real Delivery % + Candlestick Shape.
    """
    try:
        # --- A. VOLUME & DELIVERY ---
        window = Config.VOL_LOOKBACK
        vol_mean = hist['Volume'].rolling(window=window).mean().iloc[-1]
        vol_std = hist['Volume'].rolling(window=window).std().iloc[-1]
        curr_vol = hist['Volume'].iloc[-1]

        if vol_std == 0: return "Wait", "Stable Vol"
        z_score = (curr_vol - vol_mean) / vol_std

        # FETCH REAL DELIVERY
        deliv = get_delivery(ticker)

        # Threshold Checks
        is_big_vol = z_score > Config.Z_SCORE_LIMIT
        is_high_deliv = deliv > Config.DELIVERY_LIMIT

        msg = f"Z:{z_score:.1f}, Del:{deliv:.1f}%"

        # --- B. CANDLESTICK SHAPE (Rejection Logic) ---
        O = hist['Open'].iloc[-1]
        H = hist['High'].iloc[-1]
        L = hist['Low'].iloc[-1]
        C = hist['Close'].iloc[-1]

        body = abs(C - O)
        lower_wick = min(O, C) - L
        upper_wick = H - max(O, C)
        range_len = H - L

        if range_len == 0: return "Wait", "No Movement"

        # --- C. DECISION MATRIX ---

        # 1. BUY SIGNALS (Accumulation)
        if is_big_vol and is_high_deliv:
            # Sub-case: The Spring (Hammer)
            is_hammer = (lower_wick > body * 2) and (C > L + (range_len * 0.6))

            if is_hammer:
                return "🚀 BUY (Spring)", f"Price Rejection ({msg})"
            elif C > O:
                return "🚀 BUY (Strong)", f"Momentum ({msg})"
            else:
                return "⚠️ WATCH", f"Red Candle but High Deliv ({msg})"

        # 2. SELL SIGNALS (Distribution)
        if is_big_vol and is_high_deliv:
            # Sub-case: Shooting Star (Trap)
            is_shooting_star = (upper_wick > body * 2) and (C < L + (range_len * 0.4))

            if is_shooting_star:
                return "📉 SELL (Trap)", f"Bull Trap Detected ({msg})"

        # 3. FAKE SIGNALS
        if is_big_vol and not is_high_deliv:
            return "⚠️ FAKE PUMP", f"Low Delivery ({msg})"

        return "Wait", f"Normal Activity ({msg})"

    except Exception as e:
        return "Err", f"Calc Error {e}"

# ==============================================================================
# 4. MAIN CONTROLLER
# ==============================================================================
def run_scanner():
    print(f"\n{'='*60}")
    print(f"   INSTITUTIONAL SNIPER v4.1 (LIVE DATA ACTIVE)")
    print(f"{'='*60}\n")

    report_card = []

    for ticker in Config.WATCHLIST:
        print(f"Scanning {ticker}...", end="\r") # Progress indicator
        info, hist = fetch_data(ticker)
        if info is None: continue

        # 1. Run Analysis
        fund_status = analyze_fundamentals(info)
        trend_status = analyze_trend(hist)
        action, reason = check_institutional_action(hist, ticker)

        # 2. Add to Report
        report_card.append({
            "Ticker": ticker,
            "Action": action,
            "Trend": trend_status,
            "Fundamentals": fund_status,
            "Reason": reason
        })

        # Respect NSE servers (don't spam requests)
        time.sleep(1)

    # 3. Print Dashboard
    if report_card:
        print(" " * 50) # Clear progress line
        df = pd.DataFrame(report_card)
        df = df[["Ticker", "Action", "Trend", "Fundamentals", "Reason"]]
        print(df.to_string(index=False))
    else:
        print("No Data Found.")

if __name__ == "__main__":
    run_scanner()

✅ NSE Python Library Detected.

   INSTITUTIONAL SNIPER v4.1 (LIVE DATA ACTIVE)

                                                  
   Ticker Action        Trend Fundamentals                              Reason
NETWEB.NS   Wait ⚠️ Downtrend       ✅ Safe Normal Activity (Z:-0.2, Del:20.6%)


In [1]:
# ==============================================================================
# PROJECT: TECHNO-FUNDA INSTITUTIONAL SCANNER (v2.0)
# OBJECTIVE: Identify Safe Stocks with Institutional Buying Momentum
# AUTHOR: You & Gemini
# ==============================================================================

# ------------------------------------------------------------------------------
# TABLE OF CONTENTS
# ------------------------------------------------------------------------------
# 1. IMPORTS & SETUP ......... Library imports and environment checks
# 2. CONFIGURATION ........... Central control panel for all thresholds
# 3. DATA ENGINE ............. Handles yfinance and NSE connections
# 4. STRATEGY LAYERS ......... The 3-Step Filtering Logic
# 5. MAIN CONTROLLER ......... The execution loop and reporting
# ------------------------------------------------------------------------------
#




fatal: not a git repository (or any of the parent directories): .git



# Understaning notes




1. Debt-to-Equity < 1.5 (The "Mortgage" Analogy)
Think of this like buying a house.

Equity is your Down Payment (the money you actually own in the house).

Debt is the Mortgage (the money you borrowed from the bank).

If you put ₹10 Lakhs down and borrow ₹90 Lakhs, your ratio is 9.0. You are "highly leveraged." If the house price drops even a little, you are in trouble. If you put ₹50 Lakhs down and borrow ₹50 Lakhs, your ratio is 1.0. You are "safe."

The Rule (< 1.5): For every ₹1 of the shareholders' own money, the company should not owe more than ₹1.50 to the bank.

Why? In bad times (like a recession), companies with high debt go bankrupt because they must pay interest regardless of whether they made a profit. Low debt companies can survive the storm.

2. ROE > 10% (The "Savings Account" Analogy)
ROE (Return on Equity) asks: "For every ₹100 of shareholder money, how much profit did the management generate this year?"

Your Question: "I think this should be higher?"

The Answer: You are absolutely right! You are thinking like a "Growth Investor." Most top-tier investors look for 15% or 20%+.

Why use 10% then? 10% is just the "Safety Floor." It filters out the "garbage" (companies losing money or barely breaking even).

A savings account gives you ~7% risk-free.

If a risky company can't even generate 10%, it's not worth your time.

1. VOL_LOOKBACK = 20 (The Baseline)
This tells the computer how far back to look to define what is "normal" for this stock.

Layman Term: "The Monthly Average."

Why 20? There are roughly 20 trading days in a month.

Example: Imagine a coffee shop ☕. To know if today is unusually busy, you can't just look at yesterday (which might have been a holiday). You look at the average sales over the last 20 days to get a solid baseline of what "normal" looks like.

2. Z_SCORE_LIMIT = 2.0 (The Anomaly Detector)
This measures how "weird" or "extreme" today's volume is compared to that baseline.

Layman Term: "The Excitement Level."

The Math: A Z-Score of 0 is perfectly normal. A Z-Score of +2.0 means volume is significantly higher than usual (statistically rare).

Example:

Normal day: The coffee shop sells 100 cups.

Busy day (Z=1): They sell 120 cups. (Busy, but happens).

Crazy day (Z=2): They sell 160 cups. Something special is definitely happening! 😲

3. DELIVERY_LIMIT = 40.0 (The Commitment Check)
This checks what percentage of the shares traded were actually "taken home" (delivered to a demat account) versus just bought and sold rapidly (intraday trading).

Layman Term: "The Serious Buyers."

Why 40%? If people are just gambling for quick profits, they sell before the market closes (Delivery = 0%). If they are investing for the long term, they keep the shares (Delivery = 100%).

Example:

Low Delivery (20%): A crowd enters the coffee shop, looks around, takes selfies, and leaves without buying much. Lots of noise, no real business.

High Delivery (40%+): The crowd enters, and almost half of them sit down and order a full meal. They are committed customers. ✅